In [1]:
#region Chargement des packages
#from IPython import get_ipython;
#get_ipython().magic('reset -sf')
import pandas as pd
from functions.f_tools import *
from functions.f_graphicalTools import *
from Models.Prospective_conso.f_evolution_tools import *
from mycolorpy import colorlist as mcp
import numpy as np
import time
dpe_colors = ['#009900', '#33cc33', '#B3FF00', '#e6e600', '#FFB300', '#FF4D00', '#FF0000',"#000000"]

import os
if os.path.basename(os.getcwd())=="Prospective_conso":
    os.chdir('..') ## to work at project root  like in any IDE
# print(os.getcwd())
Graphic_folder = "Prospective_conso/Graphics/"
Data_folder = "Prospective_conso/data/"
pd.options.display.width = 0
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#endregion

# Tertiary building stock evolution

## *Defining an initial configuration and energy retrofit pathways*

### Table of Contents

* [1. Introduction](#intro)
* [2. Loading and filling the input data](#loading)
* [3. Simulation and representation of the results](#simulation_representation)
* [4. Input data structure](#inputdata)


## 1. Introduction <a class="anchor" id="intro"></a>

This document will help you understand the modeling we developed to represent a building stock (tertiary in this case) and energy retrofit pathways which can be applied to modify this initial state.

## 2. Loading and filling the input data <a class="anchor" id="loading"></a>

First and foremost, we load the input data, stored in an excel file. More detail on the input structure and content can be found in [Section 4](#inputdata). These input data are extracted from the different sheets and stored in sim_param dictionnary.

In [2]:
data_set_from_excel =  pd.read_excel(Data_folder+"Hypotheses_tertiary_BASIC.xlsx", None);
sim_param = extract_sim_param(data_set_from_excel,Index_names = ["Energy_source"],
                              dim_names=["Energy_source","year"])
sim_param["init_sim_stock"]=create_initial_parc(sim_param).sort_index()
sim_param["volume_variable_name"] = "surface"

For parameters indexed along year, an interpolation is performed if some data is missing (e.g. retrofit_improvement - representing the mean retrofit impact on the heating demand - starting at 15% in 2020 and reaching 30% in 2050).

In [3]:
Pameter_to_fill_along_index_year = {param : sim_param["base_index_year"] for param in ["retrofit_improvement","retrofit_change_surface","retrofit_Transition", "new_energy","new_yearly_repartition_per_Energy_source"]}
sim_param   =   complete_parameters(sim_param,Para_2_fill=Pameter_to_fill_along_index_year)

The heating need and final consumption are then calculated for each energy_source.

In [4]:
def f_Compute_conso(x,Vecteur = "total"):
    if Vecteur=="total":
        conso_unitaire = x["conso_unitaire_elec"]+x["conso_unitaire_gaz"]+x["conso_unitaire_fioul"]+x["conso_unitaire_bois"]
    else: conso_unitaire = x["conso_unitaire_"+Vecteur]
    return x["energy_need_per_surface"] * x["surface"]*x["proportion_besoin_chauffage"]*conso_unitaire
sim_param["f_Compute_conso"]=f_Compute_conso

def f_Compute_besoin(x): return x["energy_need_per_surface"] * x["surface"]*x["proportion_besoin_chauffage"]
sim_param["f_Compute_besoin"]=f_Compute_besoin


## 3. Simulation and representation of results <a class="anchor" id="simulation_representation"></a>
The simulation function calculates the surface to be retrofitted each year, and the switches from one energy source to another.

In [ ]:
sim_stock = launch_simulation(sim_param)

The evolution of heating energy needs can be represented, splitting the

In [ ]:
Var = "Besoin"
y_df = sim_stock_df.groupby(["year","Energy_source"])[Var].sum().to_frame().reset_index().\
    pivot(index='year', columns='Energy_source').loc[[year for year in range(2021,2050)],Var]/10**9
fig = MyStackedPlotly(y_df=y_df)
fig=fig.update_layout(title_text="Besoin de chaleur par mode de chauffage (en TWh)", xaxis_title="Année",yaxis_title="Besoin [TWh]")
plotly.offline.plot(fig, filename=Graphic_folder+'file.html') ## offline

## 4. Input data structure <a class="anchor" id="inputdata"></a>

All the data input are stored in the excel file data/Hypotheses_tertiaire_1D.xlsx. There are different excel sheets in the document, each of which represents input data indexed on different variables.

### 4.1. Global 0-Dim inputs
In this first sheet, global 0D values are defined:
- Initial state of the building stock: the heat need of existing buildings and the share of need due to space heating
- Parameters of newly built buildings: the heat need of new buildings and the area of newly build buildings per year
- Horizon parameters such as the start and end date of the simulation and the final retrofited share

### 4.2. Inputs indexed by Energy-source
Some values are indexed by the energy source, such as:
- The initial surface heated by each energy source
- The energy repartition by fuel, including efficiency and energy share along the year (for hybrid systems)
- The surface to be retrofitted by energy source. These surfaces are estimated to guarantee that 100% of buildings are retrofitted by 2050.


### 4.3. Input indexed by year

### 4.4. Inputs indexed by Energy-source x

### 4.5. Retrofit transition matrix

